<a href="https://colab.research.google.com/github/AI-Junction/Personalized-Medicine/blob/master/PersonalizedMedRedCancerTreatVer2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import math
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer



In [3]:
pwd = os.getcwd()
print(pwd)
dirname = os.path.join(pwd, "AllPersonalizedMedicineData")
print(dirname)


/content
/content/AllPersonalizedMedicineData


In [4]:
!pip install kaggle

    100% |████████████████████████████████| 61kB 2.2MB/s 
    100% |████████████████████████████████| 245kB 4.3MB/s 
  Running setup.py bdist_wheel for kaggle ... - \ done
  Stored in directory: /root/.cache/pip/wheels/8b/21/3b/a0076243c6ae12a6215b2da515fe06b539aee7217b406e510e
  Running setup.py bdist_wheel for python-slugify ... - done
  Stored in directory: /root/.cache/pip/wheels/e3/65/da/2045deea3098ed7471eca0e2460cfbd3fdfe8c1d6fa6fcac92
Successfully built kaggle python-slugify


In [27]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

#filename = "/content/.kaggle/kaggle.json"
#filename = "/.kaggle/kaggle.json"
filename = "kaggle.json"
#os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [28]:
!ls -lha kaggle.json

---x-wx--T 1 root root 65 Dec  2 16:04 kaggle.json


In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [0]:
!chmod 600 ~/.kaggle/kaggle.json

In [31]:
!kaggle datasets list

ref                                                           title                                                size  lastUpdated          downloadCount  
------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  
kaggle/kaggle-survey-2018                                     2018 Kaggle ML & DS Survey Challenge                  4MB  2018-11-03 22:35:07           5560  
mehdidag/black-friday                                         Black Friday                                          5MB  2018-07-25 20:49:48          14275  
jkkphys/english-wikipedia-articles-20170820-models            English Wikipedia Articles 2017-08-20 Models        924MB  2018-11-28 17:09:32             13  
center-for-policing-equity/data-science-for-good              Data Science for Good: Center for Policing Equity    43MB  2018-10-29 23:55:11           2576  
thedownhill/art-images-drawings-painting-sculpture-e

In [23]:
!kaggle competitions files -c titanic

name                   size  creationDate         
---------------------  ----  -------------------  
train.csv              60KB  2013-06-28 13:40:25  
test.csv               28KB  2013-06-28 13:40:24  
gender_submission.csv   3KB  2017-02-01 01:49:18  


In [32]:
!kaggle datasets download -d stackoverflow/stack-overflow-2018-developer-survey

 46% 9.00M/19.6M [00:00<00:00, 80.7MB/s]
100% 19.6M/19.6M [00:00<00:00, 108MB/s] 


In [42]:
!unzip stack-overflow-2018-developer-survey.zip -d stack-overflow-2018-developer-survey

Archive:  stack-overflow-2018-developer-survey.zip
  inflating: stack-overflow-2018-developer-survey/survey_results_schema.csv  
  inflating: stack-overflow-2018-developer-survey/survey_results_public.csv  


In [48]:
from subprocess import check_output
print(check_output(["ls", "/content/stack-overflow-2018-developer-survey"]).decode("utf8"))

survey_results_public.csv
survey_results_schema.csv



In [49]:
#!head ~/.kaggle/datasets/stackoverflow/stack-overflow-2018-developer-survey/survey_results_public.csv
!head /content/stack-overflow-2018-developer-survey/survey_results_public.csv

Respondent,Hobby,OpenSource,Country,Student,Employment,FormalEducation,UndergradMajor,CompanySize,DevType,YearsCoding,YearsCodingProf,JobSatisfaction,CareerSatisfaction,HopeFiveYears,JobSearchStatus,LastNewJob,AssessJob1,AssessJob2,AssessJob3,AssessJob4,AssessJob5,AssessJob6,AssessJob7,AssessJob8,AssessJob9,AssessJob10,AssessBenefits1,AssessBenefits2,AssessBenefits3,AssessBenefits4,AssessBenefits5,AssessBenefits6,AssessBenefits7,AssessBenefits8,AssessBenefits9,AssessBenefits10,AssessBenefits11,JobContactPriorities1,JobContactPriorities2,JobContactPriorities3,JobContactPriorities4,JobContactPriorities5,JobEmailPriorities1,JobEmailPriorities2,JobEmailPriorities3,JobEmailPriorities4,JobEmailPriorities5,JobEmailPriorities6,JobEmailPriorities7,UpdateCV,Currency,Salary,SalaryType,ConvertedSalary,CurrencySymbol,CommunicationTools,TimeFullyProductive,EducationTypes,SelfTaughtTypes,TimeAfterBootcamp,HackathonReasons,AgreeDisagree1,AgreeDisagree2,AgreeDisagree3,LanguageWorkedWith,LanguageDesireN

In [50]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [51]:
!kaggle competitions files -c msk-redefining-cancer-treatment

name                             size  creationDate         
-------------------------------  ----  -------------------  
training_variants.zip            24KB  2017-06-22 22:42:33  
test_variants.zip                47KB  2017-06-22 22:42:33  
training_text.zip                60MB  2017-06-22 22:42:59  
test_text.zip                    99MB  2017-06-22 22:43:13  
stage2_sample_submission.csv.7z  765B  2017-09-26 00:21:29  
stage2_test_variants.csv.7z       7KB  2017-09-26 00:21:29  
stage2_test_text.csv.7z           9MB  2017-09-26 00:21:30  
stage1_solution_filtered.csv.7z   1KB  2017-09-26 01:49:00  
stage_2_private_solution.csv.7z  592B  2017-10-13 22:04:01  


In [0]:
os.environ['KAGGLE_CONFIG_DIR'] = "/content/.kaggle"
#KAGGLE_CONFIG_DIR=/content/.kaggle

In [63]:
# to transfer files directly from kaggle competition to google drive. In below code training_variants.zip is transferred
!kaggle competitions download -c msk-redefining-cancer-treatment -f training_variants -p "/content/drive/My Drive/All ML Datasets/AllPersonalizedMedicineData"

  0% 0.00/24.2k [00:00<?, ?B/s]
100% 24.2k/24.2k [00:00<00:00, 1.44MB/s]


In [62]:

!kaggle competitions files -c msk-redefining-cancer-treatment -p "/content/drive/My Drive/All ML Datasets/AllPersonalizedMedicineData"

usage: kaggle [-h] [-v] {competitions,c,datasets,d,kernels,k,config} ...
kaggle: error: unrecognized arguments: -p


In [54]:
!ls "/content/drive/My Drive/Machine Learning/All ML Datasets"

Advertising.csv		   iris.data		  SMSSpamreadme
all_digits_mnist_test.csv  iris.names		  titanic_test.csv
cifar-10-python		   mnist_test.csv	  titanic_train.csv
cifar-10-python.tar.gz	   mnist_train.csv	  titanic.xls
diabetes.csv		   SMSSpamCollection.tsv


In [35]:
pwd = os.getcwd()
print(pwd)

/content


In [34]:
print(os.listdir())


['.config', 'c:\\Users\\echpar\\.kaggle\\kaggle.json', '.kaggle', 'kaggle.json', 'stack-overflow-2018-developer-survey.zip', 'adc.json', 'sample_data']


MessageError: ignored

In [0]:
export KAGGLE_USERNAME=cpattekar
export KAGGLE_KEY=af37bff66adaf8e646c1b35bec1ff209

SyntaxError: ignored

In [0]:
!kaggle competitions list

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 7, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.6/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.6/dist-packages/kaggle/api/kaggle_api_extended.py", line 113, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [0]:
#for f in os.listdir('../input'):
for f in os.listdir(dirname):
    size_bytes = round(os.path.getsize(os.path.join(dirname, f))/ 1000, 2)
    size_name = ["KB", "MB"]
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 2)
    print(f.ljust(20) + str(s).ljust(7) + size_name[i])


FileNotFoundError: ignored

In [0]:
train_variants_df = pd.read_csv(os.path.join(dirname, "training_variants"))
test_variants_df = pd.read_csv(os.path.join(dirname, "test_variants"))
train_text_df = pd.read_csv(os.path.join(dirname, "training_text"), sep="\|\|", engine="python", skiprows=1, names=["ID", "Text"])
test_text_df = pd.read_csv(os.path.join(dirname, "test_text"), sep="\|\|", engine="python", skiprows=1, names=["ID", "Text"])
print("Train Variant".ljust(15), train_variants_df.shape)
print("Train Text".ljust(15), train_text_df.shape)
print("Test Variant".ljust(15), test_variants_df.shape)
print("Test Text".ljust(15), test_text_df.shape)    


FileNotFoundError: ignored